In [1]:
import pandas as pd
import numpy as np
from scipy.stats import linregress
from sklearn.preprocessing import MinMaxScaler

# 데이터 불러오기 (파일 경로에 맞게 수정 필요)
df = pd.read_csv("/Users/sangrimkoo/Desktop/프로젝트/사용하는데이터/VIP데이터.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/sangrimkoo/Desktop/프로젝트/사용하는데이터/VIP데이터.csv'

In [ ]:
len(df)

11460

In [ ]:
df = df.sort_values(by=['발급회원번호', '기준년월'])

In [ ]:
def calc_slope_6m(series):
    # 데이터가 2개 미만이거나 전부 0이면 계산 불가 (0 반환)
    if len(series) < 2 or np.sum(series) == 0:
        return 0
    
    y = series.values
    x = np.arange(len(y))
    
    # 값이 6개월 내내 똑같으면 기울기 0
    if np.all(y == y[0]): return 0
    
    # 선형 회귀로 기울기 계산
    slope, _, _, _, _ = linregress(x, y)
    
    # 계산 오류(NaN)가 나면 0으로 처리
    if np.isnan(slope): return 0
    
    return slope

In [ ]:
print("고객별 추세(기울기) 계산 중... (시간이 좀 걸릴 수 있어!)")

# 각 지표별로 6개월 기울기 계산
df['Slope_Spend'] = df.groupby('발급회원번호')['이용금액_신용_B0M'].transform(calc_slope_6m)

# 잔액 컬럼 이름 확인 필요 (보통 '잔액_합계_B0M' 또는 '잔액_B0M')
balance_col = '잔액_B0M' if '잔액_B0M' in df.columns else df.columns[df.columns.str.contains('잔액')][0]
df['Slope_Balance'] = df.groupby('발급회원번호')[balance_col].transform(calc_slope_6m)

df['Slope_Count'] = df.groupby('발급회원번호')['이용건수_신용_B0M'].transform(calc_slope_6m)

# 가장 최근 월(12월) 데이터만 추출 (현재 시점 기준 분석)
target_month = df['기준년월'].max()
df_final = df[df['기준년월'] == target_month].copy()

print(f"분석 대상 월: {target_month}")

고객별 추세(기울기) 계산 중... (시간이 좀 걸릴 수 있어!)
분석 대상 월: 201812


In [ ]:
scaler = MinMaxScaler()

def convert_slope_to_score(df_in, col_name):
    # 감소(음수)하는 경우만 추출하여 양수로 변환 (감소폭이 클수록 높은 점수)
    # 증가하거나 변동 없는 경우(0 이상)는 0점으로 처리 (위험하지 않으니까!)
    negative_slopes = df_in[col_name].apply(lambda x: -x if x < 0 else 0)
    
    # 0~1 사이로 점수 압축 (MinMax Scaling)
    return scaler.fit_transform(negative_slopes.values.reshape(-1, 1)).flatten()

# 각 지표별 점수 계산
score_spend = convert_slope_to_score(df_final, 'Slope_Spend')
score_balance = convert_slope_to_score(df_final, 'Slope_Balance')
score_count = convert_slope_to_score(df_final, 'Slope_Count')

In [ ]:
# 가중치 설정 (합계 100)
W_SPEND = 40   # 소비 감소 비중 (제일 중요!)
W_BALANCE = 30 # 잔액 감소 비중
W_COUNT = 20   # 빈도 감소 비중
W_RISK = 10    # 리스크 비중

# 리스크 점수 (연체나 거절 이력 있으면 1, 없으면 0)
has_risk = np.where(
    (df_final.get('연체잔액_B0M', 0) > 0) | 
    (df_final.get('승인거절건수_B0M', 0) > 0), 
    1, 0
)

# 최종 점수 계산
df_final['Churn_Score'] = (
    (score_spend * W_SPEND) + 
    (score_balance * W_BALANCE) + 
    (score_count * W_COUNT) + 
    (has_risk * W_RISK)
)

# 보기 좋게 소수점 첫째 자리까지만 남김
df_final['Churn_Score'] = df_final['Churn_Score'].round(1)

In [ ]:
print(f"\n[Churn_Score 생성 완료]")
print(f"전체 고객 수: {len(df_final)}명")
print(f"점수 평균: {df_final['Churn_Score'].mean():.2f}점")
print(f"점수 최대값: {df_final['Churn_Score'].max()}점")

print("\n[상위 5명 데이터 확인]")
cols_view = ['발급회원번호', 'Slope_Spend', 'Slope_Balance', 'Churn_Score']
print(df_final[cols_view].sort_values('Churn_Score', ascending=False).head())


[Churn_Score 생성 완료]
전체 고객 수: 1910명
점수 평균: 8.97점
점수 최대값: 63.7점

[상위 5명 데이터 확인]
         발급회원번호    Slope_Spend  Slope_Balance  Churn_Score
10325  SYN_3968 -243906.428571 -117226.657143         63.7
10596  SYN_5296 -206826.314286 -381888.542857         55.4
11267  SYN_9037 -229686.857143 -119831.371429         54.9
10230  SYN_3442 -168908.285714 -186469.485714         51.4
9864   SYN_1527 -186951.657143 -241112.200000         49.6


In [ ]:
#### 모델을 만들고 shap를 한다

In [ ]:
import xgboost as xgb
import shap
import matplotlib.pyplot as plt

# 학습에 방해되는 컬럼(정답 만들 때 쓴 재료들 + ID) 제거
drop_cols = ['Churn_Score', 'Slope_Spend', 'Slope_Balance', 'Slope_Count', 
             '발급회원번호', '기준년월', 'VIP등급코드', 'Target', 'Churn_Refined']

# 실제 데이터에 있는 것만 골라서 뺌
existing_drop_cols = [c for c in drop_cols if c in df_final.columns]

# X: 문제 (숫자 데이터만), y: 정답 (점수)
X = df_final.drop(columns=existing_drop_cols).select_dtypes(include=[np.number])
X = X.fillna(0) # 빈칸은 0으로 채움
y = df_final['Churn_Score']

print("데이터 준비 완료!")

/usr/local/Caskroom/miniconda/base/envs/myenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


데이터 준비 완료!


In [ ]:
# 회귀 모델(XGBoost Regressor) 생성
# n_estimators=100: 공부를 100번 반복해라
model = xgb.XGBRegressor(n_estimators=100, random_state=42)

print("모델 학습 중...")
model.fit(X, y)
print("학습 완료! AI가 이제 점수를 예측할 수 있어.")

모델 학습 중...
학습 완료! AI가 이제 점수를 예측할 수 있어.


In [ ]:
feature_importance = model.feature_importances_
sorted_idx = np.argsort(feature_importance)

print("\n[Top 20 Features and their Importance Scores]")
print("="*50)
for idx in sorted_idx[::-1][:20]:
    print(f"{X.columns[idx]}: {feature_importance[idx]:.4f}")

print("\n[Churn Score Statistics]")
print("="*50)
print(df_final['Churn_Score'].describe())


[Top 20 Features and their Importance Scores]
이용건수_할부_R6M: 0.1065
이용금액_할부_R3M: 0.0696
평잔_RV일시불_해외_6M: 0.0455
이용금액_할부_R6M: 0.0333
정상청구원금_B5M: 0.0211
잔액_카드론_B4M: 0.0192
포인트_마일리지_건별_B0M: 0.0191
카드론이용월수_누적: 0.0167
평잔_RV일시불_해외_3M: 0.0154
이용금액_할부_B0M: 0.0152
이용금액_부분무이자_R12M: 0.0146
할부금액_유이자_3M_R12M: 0.0120
이용금액_신용_B0M: 0.0118
여유_Pet이용금액: 0.0109
이용개월수_할부_유이자_R6M: 0.0106
평잔_카드론_3M: 0.0104
RV_최대잔액_R12M: 0.0102
이용건수_선결제_R6M: 0.0090
평잔_6M: 0.0090
이용메뉴건수_ARS_R6M: 0.0088

[Churn Score Statistics]
count    1910.000000
mean        8.968901
std         9.084385
min         0.000000
25%         1.900000
50%         6.300000
75%        13.375000
max        63.700000
Name: Churn_Score, dtype: float64


In [ ]:
# -------------------------------------------------------
# 8. 결과 파일 저장하기 (CSV)
# -------------------------------------------------------

# (1) 고객별 이탈 점수 및 등급표 저장
df_final.to_csv("VIP_고객_이탈위험점수.csv", index=False, encoding='utf-8-sig')

# (2) 변수 중요도(Feature Importance) 저장
# 보기 좋게 표(DataFrame)로 만들어서 저장할게
importance_df = pd.DataFrame({
    '변수명': X.columns,
    '중요도': feature_importance
})

# 중요도가 높은 순서대로 정렬하기
importance_df = importance_df.sort_values(by='중요도', ascending=False)

# 파일로 저장
importance_df.to_csv("이탈예측_변수중요도.csv", index=False, encoding='utf-8-sig')

print("\n🎉 파일 저장이 완료되었어!")
print("📂 1. VIP_고객_이탈위험점수.csv (고객별 점수표)")
print("📂 2. 이탈예측_변수중요도.csv (중요한 변수 순위)")


🎉 파일 저장이 완료되었어!
📂 1. VIP_고객_이탈위험점수.csv (고객별 점수표)
📂 2. 이탈예측_변수중요도.csv (중요한 변수 순위)
